# Heatmap Visualization Example

This notebook demonstrates the heatmap feature of iops-profiler, which provides a time-series heatmap view of I/O operations.

The heatmap shows:
- **X-axis**: Time (runtime in seconds)
- **Y-axis**: Operation size (bytes, log scale)
- **Color**: Either the number of operations or total bytes transferred

This visualization helps identify I/O patterns over time, showing when different sizes of operations occur during program execution.

In [ ]:
# Load the iops-profiler extension
%load_ext iops_profiler

## Example 1: Simple I/O with Varying Sizes

Let's create a workload that performs I/O operations of different sizes over time:

In [ ]:
%%iops --heatmap
import tempfile
import os
import time

# Create a temporary directory for our test files
test_dir = tempfile.mkdtemp()

try:
    # Phase 1: Small writes (1 KB each)
    for i in range(5):
        with open(os.path.join(test_dir, f'small_{i}.txt'), 'w') as f:
            f.write('x' * 1024)  # 1 KB
        time.sleep(0.05)  # Small delay to spread operations over time
    
    # Phase 2: Medium writes (10 KB each)
    for i in range(5):
        with open(os.path.join(test_dir, f'medium_{i}.txt'), 'w') as f:
            f.write('y' * (10 * 1024))  # 10 KB
        time.sleep(0.05)
    
    # Phase 3: Large writes (100 KB each)
    for i in range(5):
        with open(os.path.join(test_dir, f'large_{i}.txt'), 'w') as f:
            f.write('z' * (100 * 1024))  # 100 KB
        time.sleep(0.05)
    
    # Phase 4: Mixed reads - read back all files in order
    for size_category in ['small', 'medium', 'large']:
        for i in range(5):
            filepath = os.path.join(test_dir, f'{size_category}_{i}.txt')
            if os.path.exists(filepath):
                with open(filepath, 'r') as f:
                    _ = f.read()
                time.sleep(0.03)

finally:
    # Cleanup
    import shutil
    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)

The heatmap above shows:

1. **Left plot (Operation Count)**: How many I/O operations occurred in each time/size bin
2. **Right plot (Total Bytes)**: How much data was transferred in each time/size bin

You can see distinct phases:
- Early time: Small operations (1 KB writes)
- Middle time: Medium operations (10 KB writes)
- Later time: Large operations (100 KB writes)
- Final time: Mixed reads of all sizes

## Example 2: Bursty I/O Pattern

Let's create a workload with bursts of activity at different scales:

In [ ]:
%%iops --heatmap
import tempfile
import os
import time

test_dir = tempfile.mkdtemp()

try:
    # Burst 1: Many small operations
    for i in range(20):
        with open(os.path.join(test_dir, f'burst1_{i}.txt'), 'w') as f:
            f.write('a' * 512)  # 512 bytes
    
    # Pause
    time.sleep(0.2)
    
    # Burst 2: Few large operations
    for i in range(3):
        with open(os.path.join(test_dir, f'burst2_{i}.txt'), 'w') as f:
            f.write('b' * (200 * 1024))  # 200 KB
    
    # Pause
    time.sleep(0.2)
    
    # Burst 3: Medium-sized operations
    for i in range(10):
        with open(os.path.join(test_dir, f'burst3_{i}.txt'), 'w') as f:
            f.write('c' * (5 * 1024))  # 5 KB

finally:
    import shutil
    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)

This heatmap shows a bursty pattern with clear gaps between activity phases. The color intensity helps identify which phases had the most activity or transferred the most data.

## Comparing with Histogram View

For comparison, here's the same workload with the histogram view:

In [ ]:
%%iops --histogram
import tempfile
import os
import time

test_dir = tempfile.mkdtemp()

try:
    # Same workload as Example 1
    for i in range(5):
        with open(os.path.join(test_dir, f'small_{i}.txt'), 'w') as f:
            f.write('x' * 1024)
        time.sleep(0.05)
    
    for i in range(5):
        with open(os.path.join(test_dir, f'medium_{i}.txt'), 'w') as f:
            f.write('y' * (10 * 1024))
        time.sleep(0.05)
    
    for i in range(5):
        with open(os.path.join(test_dir, f'large_{i}.txt'), 'w') as f:
            f.write('z' * (100 * 1024))
        time.sleep(0.05)

finally:
    import shutil
    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)

The histogram shows the distribution of operation sizes but doesn't reveal when they occurred. The heatmap adds the temporal dimension, making it easier to:

- Identify when different I/O patterns occur during execution
- Spot bursts or gaps in I/O activity
- Correlate I/O behavior with application phases
- Debug performance issues related to I/O timing